In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import pandas as pd
import time
from datetime import datetime

In [13]:
df=pd.read_excel('C:/Users/stefa/Desktop/Romania_RNN.xlsx')
df

,Lat,Lon,Depth,Mag,An,Previous
0,46.5000,27.3000,9.9,3.2,1900,0
1,46.0000,21.2000,9.9,4.5,1900,0
2,46.5000,27.3000,9.9,5.5,1900,0
3,44.3000,26.3000,9.9,2.6,1900,0
4,45.0000,24.1000,9.9,4.2,1900,0
...,...,...,...,...,...,...
30499,46.0443,22.8680,5.9,1.9,2021,35
30500,46.8633,22.8872,5.0,1.9,2021,35
30501,45.6367,27.9550,20.3,1.9,2021,35
30502,45.7725,26.6335,74.3,3.0,2021,35


In [23]:
col1 = df[['Previous','Lat','Lon','Depth']]
col2 = df['Mag']
#Convert to Numpy array
InputX1 = col1.to_numpy()
InputY1 = col2.to_numpy()
print(InputX1)

[[  0.      46.5     27.3      9.9   ]
 [  0.      46.      21.2      9.9   ]
 [  0.      46.5     27.3      9.9   ]
 ...
 [ 35.      45.6367  27.955   20.3   ]
 [ 35.      45.7725  26.6335  74.3   ]
 [ 35.      45.5518  26.4571 110.7   ]]


In [15]:
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 2 #Number of input features
Yfeatures = 1 #Number of input features
samples = 30000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))

Mininum values: [0. 0.]
Maximum values: [ 37.  218.4]


In [16]:
batch_size = 1000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 3000
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]

In [17]:
learning_rate = 0.01
training_iterations = 1000
display_iterations = 200

In [18]:
tf.compat.v1.disable_eager_execution()
#Input
X = tf.compat.v1.placeholder(tf.float32,shape=(None,Xfeatures))
#tf.compat.v1.placeholder(shape=[None, 2], dtype=tf.float32)
#Output
Y = tf.compat.v1.placeholder(tf.float32)

In [19]:
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [20]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [21]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()

In [22]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))

Training loss: [8.744412]
Training loss is: [8.06355] at itertion: 0
Validation loss is: [2.9987285] at itertion: 0
Training loss is: [1.9957808] at itertion: 200
Validation loss is: [2.2262611] at itertion: 200
Training loss is: [1.9901286] at itertion: 400
Validation loss is: [2.2470756] at itertion: 400
Training loss is: [1.9901288] at itertion: 600
Validation loss is: [2.2471018] at itertion: 600
Training loss is: [1.9901286] at itertion: 800
Validation loss is: [2.2471015] at itertion: 800
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [1.9901288]
Final validation loss: [2.2471015]
